USING VGG MODEL ,  WITH UNDERSAMPLING AND DATA AUGMENTATION


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import io
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/fer2013.csv")


In [ ]:
def convert_pixels_to_img(pixels):
    test_image = pixels.reshape(48,48,1)
    return test_image

def split_pixels(string):
    splitted = np.array(string.split(),'int')
    return splitted 

def change_to_categorical(sample):
    return keras.utils.to_categorical(sample,num_classes=7)


In [ ]:
dataset['pixels'] = dataset['pixels'].apply(split_pixels)
dataset['label'] = dataset['emotion'].apply(change_to_categorical)
dataset['length'] = dataset['pixels'].apply(len)

dataset = dataset[dataset.length == 2304]
dataset['image'] = dataset['pixels'].apply(convert_pixels_to_img)

In [ ]:
grouped_dataset = dataset.groupby(dataset.Usage)
training_dataset = grouped_dataset.get_group("Training")
dev_dataset = grouped_dataset.get_group("PublicTest")
test_dataset = grouped_dataset.get_group("PrivateTest")

In [ ]:
training_dataset = training_dataset.loc[training_dataset["emotion"]!=1]
dev_dataset = dev_dataset.loc[dev_dataset["emotion"]!=1]
test_dataset = test_dataset.loc[test_dataset["emotion"]!=1]
def balancingDataset(x):
  new_df=training_dataset.loc[dataset['emotion']==x]
  new_df=new_df[:3000]
  return new_df
df_0=balancingDataset(0)
df_2=balancingDataset(2)
df_3=balancingDataset(3)
df_4=balancingDataset(4)
df_5=balancingDataset(5)
df_6=balancingDataset(6)


training_dataset=pd.concat([df_0,df_2,df_3,df_4,df_5,df_6])
training_dataset

,emotion,pixels,Usage,label,length,image
0,0,"[70, 80, 82, 72, 58, 58, 60, 63, 54, 58, 60, 4...",Training,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[70], [80], [82], [72], [58], [58], [60], [6..."
1,0,"[151, 150, 147, 155, 148, 133, 111, 140, 170, ...",Training,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[151], [150], [147], [155], [148], [133], [1..."
10,0,"[30, 24, 21, 23, 25, 25, 49, 67, 84, 103, 120,...",Training,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[30], [24], [21], [23], [25], [25], [49], [6..."
22,0,"[123, 125, 124, 142, 209, 226, 234, 236, 231, ...",Training,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[123], [125], [124], [142], [209], [226], [2..."
23,0,"[8, 9, 14, 21, 26, 32, 37, 46, 52, 62, 72, 70,...",Training,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[8], [9], [14], [21], [26], [32], [37], [46]..."
...,...,...,...,...,...,...
17073,6,"[101, 105, 136, 141, 165, 167, 164, 170, 170, ...",Training,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[101], [105], [136], [141], [165], [167], [1..."
17075,6,"[116, 131, 120, 84, 43, 55, 58, 61, 61, 66, 70...",Training,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[116], [131], [120], [84], [43], [55], [58],..."
17081,6,"[255, 255, 254, 254, 254, 254, 254, 255, 255, ...",Training,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[255], [255], [254], [254], [254], [254], [2..."
17082,6,"[50, 52, 50, 49, 44, 51, 51, 41, 33, 39, 43, 4...",Training,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[50], [52], [50], [49], [44], [51], [51], [4..."


In [ ]:
dev_dataset

,emotion,pixels,Usage,label,length,image
28709,0,"[254, 254, 254, 254, 254, 249, 255, 160, 2, 58...",PublicTest,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[254], [254], [254], [254], [254], [249], [2..."
28711,4,"[69, 118, 61, 60, 96, 121, 103, 87, 103, 88, 7...",PublicTest,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2304,"[[[69], [118], [61], [60], [96], [121], [103],..."
28712,6,"[205, 203, 236, 157, 83, 158, 120, 116, 94, 86...",PublicTest,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[205], [203], [236], [157], [83], [158], [12..."
28713,3,"[87, 79, 74, 66, 74, 96, 77, 80, 80, 84, 83, 8...",PublicTest,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2304,"[[[87], [79], [74], [66], [74], [96], [77], [8..."
28714,3,"[235, 233, 223, 109, 34, 37, 34, 31, 28, 38, 5...",PublicTest,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2304,"[[[235], [233], [223], [109], [34], [37], [34]..."
...,...,...,...,...,...,...
32293,4,"[178, 176, 172, 173, 173, 174, 176, 173, 166, ...",PublicTest,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2304,"[[[178], [176], [172], [173], [173], [174], [1..."
32294,3,"[25, 34, 42, 44, 42, 47, 57, 59, 59, 58, 54, 5...",PublicTest,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2304,"[[[25], [34], [42], [44], [42], [47], [57], [5..."
32295,4,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",PublicTest,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2304,"[[[255], [255], [255], [255], [255], [255], [2..."
32296,4,"[33, 25, 31, 36, 36, 42, 69, 103, 132, 163, 17...",PublicTest,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2304,"[[[33], [25], [31], [36], [36], [42], [69], [1..."


In [ ]:
test_dataset

,emotion,pixels,Usage,label,length,image
32298,0,"[170, 118, 101, 88, 88, 75, 78, 82, 66, 74, 68...",PrivateTest,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[170], [118], [101], [88], [88], [75], [78],..."
32299,5,"[7, 5, 8, 6, 7, 3, 2, 6, 5, 4, 4, 5, 7, 5, 5, ...",PrivateTest,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]",2304,"[[[7], [5], [8], [6], [7], [3], [2], [6], [5],..."
32300,6,"[232, 240, 241, 239, 237, 235, 246, 117, 24, 2...",PrivateTest,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[232], [240], [241], [239], [237], [235], [2..."
32301,4,"[200, 197, 149, 139, 156, 89, 111, 58, 62, 95,...",PrivateTest,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2304,"[[[200], [197], [149], [139], [156], [89], [11..."
32302,2,"[40, 28, 33, 56, 45, 33, 31, 78, 152, 194, 200...",PrivateTest,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[40], [28], [33], [56], [45], [33], [31], [7..."
...,...,...,...,...,...,...
35882,6,"[50, 36, 17, 22, 23, 29, 33, 39, 34, 37, 37, 3...",PrivateTest,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",2304,"[[[50], [36], [17], [22], [23], [29], [33], [3..."
35883,3,"[178, 174, 172, 173, 181, 188, 191, 194, 196, ...",PrivateTest,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2304,"[[[178], [174], [172], [173], [181], [188], [1..."
35884,0,"[17, 17, 16, 23, 28, 22, 19, 17, 25, 26, 20, 2...",PrivateTest,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2304,"[[[17], [17], [16], [23], [28], [22], [19], [1..."
35885,3,"[30, 28, 28, 29, 31, 30, 42, 68, 79, 81, 77, 6...",PrivateTest,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2304,"[[[30], [28], [28], [29], [31], [30], [42], [6..."


In [ ]:
model = keras.Sequential()
model.add(layers.Conv2D(input_shape=(48,48,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=4096,activation="relu"))
model.add(layers.Dense(units=4096,activation="relu"))
model.add(layers.Dense(units=7, activation="softmax"))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 256)       2

In [ ]:
x_train = training_dataset['image']
y_train = training_dataset['label']
x_test =  dev_dataset['image']
y_test = dev_dataset['label']

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()


x_train = np.stack(x_train,axis=0)
y_train = np.stack(y_train,axis=0)
x_test = np.stack(x_test,axis=0)
y_test = np.stack(y_test,axis=0)

In [ ]:
datagen = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        shear_range = 10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)


In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Model/vgg_undersampling.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
model.fit(datagen.flow(x_train, y_train, batch_size=32),batch_size=32,epochs=100,verbose=1,callbacks=[checkpointer,lr_reducer,early_stopper],validation_data=(x_test, y_test))


Epoch 1/100
563/563 [==============================] - 46s 26ms/step - loss: 1.8204 - accuracy: 0.1756 - val_loss: 1.7602 - val_accuracy: 0.2454

Epoch 00001: val_loss improved from inf to 1.76020, saving model to /content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Model/vgg_undersampling.h5
Epoch 2/100
563/563 [==============================] - 14s 25ms/step - loss: 1.7774 - accuracy: 0.2128 - val_loss: 1.7298 - val_accuracy: 0.2559

Epoch 00002: val_loss improved from 1.76020 to 1.72979, saving model to /content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Model/vgg_undersampling.h5
Epoch 3/100
563/563 [==============================] - 14s 25ms/step - loss: 1.7429 - accuracy: 0.2499 - val_loss: 1.7493 - val_accuracy: 0.2734

Epoch 00003: val_loss did not improve from 1.72979
Epoch 4/100
563/563 [==============================] - 14s 25ms/step - loss: 1.6803 - accuracy: 0.2938 - val_loss: 1.5852 - val_accuracy: 0.3937

Epoch 00004: val_loss improved from 1.72979 to 1.

In [ ]:
x_test =  test_dataset['image']
y_test = test_dataset['label']

#Pandas dataframe to numpy
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

#numpy array reshape to fit into model
x_test = np.stack(x_test,axis=0)
y_test = np.stack(y_test,axis=0)

In [ ]:
model.evaluate(x_test,y_test)


111/111 [==============================] - 1s 10ms/step - loss: 1.1587 - accuracy: 0.6531


[1.1586873531341553, 0.6530843377113342]